In [3]:
from keras.layers import *
from keras.layers import LSTM, RepeatVector
from keras import Model, objectives
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras import Sequential
import keras as kr

def create_lstm_vae(input_dim,
                    timesteps,
                    batch_size,
                    intermediate_dim=75,
                    latent_dim=7,
                    epsilon_std=1.):
 
    x = Input(shape=(timesteps, input_dim,))

    print(K.shape(x))
    h = LSTM(intermediate_dim)(x)

    z_mean = Dense(latent_dim)(h)
    z_log_sigma = Dense(latent_dim)(h)

    def sampling(args):
        z_mean, z_log_sigma = args
        epsilon = K.random_normal(shape=(375, latent_dim),
                                  mean=0., stddev=epsilon_std)
        return z_mean + z_log_sigma * epsilon

    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])

    decoder_h = LSTM(intermediate_dim, return_sequences=True)
    decoder_mean = LSTM(input_dim, return_sequences=True)

    h_decoded = RepeatVector(timesteps)(z)
    h_decoded = decoder_h(h_decoded)

    x_decoded_mean = decoder_mean(h_decoded)

    vae = Model(x, x_decoded_mean)
    
    encoder = Model(x, z_mean)

    decoder_input = Input(shape=(latent_dim,))

    _h_decoded = RepeatVector(timesteps)(decoder_input)
    _h_decoded = decoder_h(_h_decoded)

    _x_decoded_mean = decoder_mean(_h_decoded)
    generator = Model(decoder_input, _x_decoded_mean)

    def vae_loss(x, x_decoded_mean):
        xent_loss = objectives.mse(x, x_decoded_mean)
        kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma))
        loss = xent_loss + kl_loss
        return loss

    print("Compiling")
    vae.compile(optimizer='rmsprop', loss=vae_loss, metrics=['accuracy'])
    vae.summary()
    return vae, encoder, generator

print("Done")


Done


In [5]:
def read_data(data_path):
    data = pd.read_csv(data_path)
    return data

boat_csv = read_data("Data/Boat_data.csv")
boat_csv = boat_csv.drop(columns=["Unnamed: 0"])
boat_csv = boat_csv.drop(boat_csv.index[-39:])
scaler = StandardScaler()
normal_data = scaler.fit_transform(boat_csv)

boat_csv = read_data("Data/Anomalous_boat_data.csv")
boat_csv = boat_csv.drop(columns=["Unnamed: 0", "heading"])
boat_csv = boat_csv.drop(boat_csv.index[-20:])    
scaler = StandardScaler()
anomalous_data = scaler.fit_transform(boat_csv)

print("Done")

Done


In [6]:
batch_size, timesteps = 75 , 75
interval = 15
def prepare_sequences(data):
    samples = []
    for i in range(0,data.shape[0] - batch_size, interval ):
        sample = data[i:i+batch_size]	
        samples.append(sample)
        
    sequences = np.array(samples)
    
    # Batch size (Number of samples time steps and number of features
    trainX = np.reshape(sequences, (len(sequences), batch_size, 7))
    
    return trainX

trainX_nominal = prepare_sequences(normal_data) 
print(trainX_nominal.shape)

input_length = trainX_nominal.shape[0]
trainX_anomalous = prepare_sequences(anomalous_data)
print(trainX_anomalous.shape)  


(375, 75, 7)
(435, 75, 7)


In [8]:
epochs = 1
vae, encoder, generator = create_lstm_vae(input_dim=7,timesteps=timesteps,
                                          batch_size=375,
                                          intermediate_dim=75,latent_dim=7)
print("Done")


Tensor("Shape_1:0", shape=(3,), dtype=int32)


Compiling
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 75, 7)        0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, 75)           24900       input_3[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 7)            532         lstm_4[0][0]                     
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 7)            532         lstm_4[0][0]                     
__________________________________________________________________________________________________


In [25]:

vae.fit(x=trainX_nominal, y=trainX_nominal, epochs=epochs, 
        batch_size=batch_size)

vae.save("Models/Nominal_LSTM_VAE.model")
print("MODEL SAVED")


Epoch 1/1


InvalidArgumentError: Incompatible shapes: [375] vs. [75]
	 [[{{node training_3/RMSprop/gradients/loss_5/lstm_18_loss/mul_1_grad/BroadcastGradientArgs}}]]

In [ ]:

autoencoded_nominal = generator.predict(encoder.predict(trainX_nominal))

autoencoded_anomalous = generator.predict(encoder.predict(trainX_anomalous))

anomalous_sequences = []
reconstr_error_nominal_sequences, reconstr_error_anomalous_sequences = [], []
MUs, SIGMAs = [], []
window = batch_size
interval = interval
cont = 0

for i in autoencoded_nominal:
    reconstr_error_nominal_sequences.append(abs(normal_data[cont:cont+window] - i))
    cont += interval
    MUs.append(np.average(reconstr_error_nominal_sequences[-1], axis=0))
    SIGMAs.append(np.cov(reconstr_error_nominal_sequences[-1], rowvar=False))

cont = 0
for j in autoencoded_anomalous:
    reconstr_error_anomalous_sequences.append(abs(anomalous_data[cont:cont+window] - j))
    cont += interval

print(len(reconstr_error_nominal_sequences), len(reconstr_error_nominal_sequences[0]))
print(len(MUs), MUs[0])
print("Done") 


In [ ]:
def calculate_anomaly_score_sequence(i, element):
    scores = []
    for j, elem in enumerate(element):
        scores.append(np.dot(np.transpose(element[j]-MUs[i]), 
                             np.dot(SIGMAs[i], (element[j]-MUs[i]))))
    return np.average(scores)

anomaly_scores = []
for i,element in enumerate(reconstr_error_anomalous_sequences[:-60]):
    anomaly_scores.append(calculate_anomaly_score_sequence(i, element))

anomaly_scores_nominal = []
for i, element in enumerate(reconstr_error_nominal_sequences):
    anomaly_scores_nominal.append(calculate_anomaly_score_sequence(i, element))

print(len(anomaly_scores), len(anomaly_scores_nominal))


In [ ]:

plt.plot(anomaly_scores)
plt.show()


In [53]:
def detectOutliers(x, outlierConstant):
    a = np.array(x)
    upper_quartile = np.percentile(a, 75)
    lower_quartile = np.percentile(a, 25)
    IQR = (upper_quartile - lower_quartile) * outlierConstant
    quartileSet = (lower_quartile - IQR, upper_quartile + IQR)
    resultList = []
    outlierList = []
    
    list = a.tolist()
    for y in range(len(list)):
        if list[y] >= quartileSet[0] and list[y] <= quartileSet[1]:
            resultList.append(list[y])
        else:
            outlierList.append((y,list[y]))
            resultList.append(list[y-1])
    return resultList, outlierList

nominal_without_outliers, outlier_list = detectOutliers(anomaly_scores_nominal,
                                                        outlierConstant=1)

anomalous_without_outliers, outliers_anomalous_list = detectOutliers(anomaly_scores,
                                                                     outlierConstant=0.001)

print("Done")


In [54]:

#PLOT THE GRAPHS AND OUTLINE THE POINTS OF THE SUSPECTED ANOMALIES

boat_csv = read_data("Data/Boat_data.csv")
boat_csv = boat_csv.drop(columns=["Unnamed: 0"])

plt.plot(boat_csv["G_Lon"], boat_csv["G_Lat"])
plt.title("Nominal anoamalies points LSTM_AE")
for i in outlier_list:
    anomaly_position = i[0]*interval 
    plt.plot(boat_csv["G_Lon"][anomaly_position],boat_csv["G_Lat"][anomaly_position], 'bo')
    print()
    
plt.show()


an_csv = read_data("Data/Anomalous_boat_data.csv")
an_csv = an_csv.drop(columns=["Unnamed: 0"])

plt.plot(an_csv["longitude"], an_csv["latitude"])
plt.title("Nominal anoamalies points LSTM_AE")
for i in outliers_anomalous_list:
    anomaly_position = i[0] * interval 
    plt.plot(an_csv["longitude"][anomaly_position:anomaly_position+window],
             an_csv["latitude"][anomaly_position:anomaly_position+window], 'bo')

plt.show()


In [24]:
# MU SIGMA FOR THE NOMINAL ERROR
MU = np.average(reconstr_nominal, axis=0)
SIGMA = np.cov(reconstr_nominal, rowvar=False)

anomaly_scores = []

def calculate_anomaly_score(i):
    return np.dot(np.transpose(i-MU), np.dot(SIGMA, (i-MU)))

for i in reconstruct_anomalous:
    anomaly_scores.append(calculate_anomaly_score(i))

anomaly_scores_nominal = []
for i in reconstr_nominal:
    anomaly_scores_nominal.append(calculate_anomaly_score(i))

plt.plot(anomaly_scores)
plt.xlabel("OBSERVATIONS")
plt.ylabel("ANOMALY SCORE")
plt.title("ANOMALOUS DATASET LSTM_VAE")
plt.show()

plt.plot(anomaly_scores_nominal)
plt.xlabel("OBSERVATIONS")
plt.ylabel("ANOMALY SCORE")
plt.title("NOMINAL DATASET LSTM_VAE")
plt.show()

In [21]:
def detectOutliers(x, outlierConstant):
    a = np.array(x)
    upper_quartile = np.percentile(a, 75)
    lower_quartile = np.percentile(a, 25)
    IQR = (upper_quartile - lower_quartile) * outlierConstant
    quartileSet = (lower_quartile - IQR, upper_quartile + IQR)
    resultList = []
    outlierList = []
    
    list = a.tolist()
    for y in range(len(list)):
        if list[y] >= quartileSet[0] and list[y] <= quartileSet[1]:
            resultList.append(list[y])
        else:
            outlierList.append((y,list[y]))
            resultList.append(list[y-1])
    return resultList, outlierList

nominal_without_outliers, outlier_list = detectOutliers(anomaly_scores_nominal,
                                                        outlierConstant=7)

anomalous_without_outliers, outliers_anomalous_list = detectOutliers(anomaly_scores,
                                                                     outlierConstant=10)



print(len(outlier_list))


24


In [ ]:

#PLOT THE GRAPHS AND OUTLINE THE POINTS OF THE SUSPECTED ANOMALIES

boat_csv = read_data("Data/Boat_data.csv")
boat_csv = boat_csv.drop(columns=["Unnamed: 0"])

plt.plot(boat_csv["G_Lon"], boat_csv["G_Lat"])
plt.title("Nominal anoamalies points LSTM_VAE")
for i in outlier_list:
    plt.plot(boat_csv["G_Lon"][i[0]],boat_csv["G_Lat"][i[0]], 'bo')
    
    print()
    
plt.show()


an_csv = read_data("Data/Anomalous_boat_data.csv")
an_csv = an_csv.drop(columns=["Unnamed: 0"])

plt.plot(an_csv["longitude"], an_csv["latitude"])
plt.title("Nominal anoamalies points LSTM_VAE")
for i in outliers_anomalous_list:
    plt.plot(an_csv["longitude"][i[0]],an_csv["latitude"][i[0]], 'bo')

plt.show()


In [27]:
autoenc_df = pd.DataFrame(autoencoded_nominal, columns= boat_csv.columns)
print(autoenc_df)


plt.plot(boat_csv["Speed"][:-114])
plt.plot(autoenc_df['Speed'])
plt.show()

print("End")

         Speed   Degrees  Accelleration       M0C       M1C     G_Lat  \
0     0.002750  0.007589       0.001174 -0.004248  0.001974 -0.015252   
1     0.007234  0.017179      -0.000572 -0.009060  0.007041 -0.043760   
2     0.015100  0.028747      -0.003854 -0.013901  0.013474 -0.083505   
3     0.027534  0.042639      -0.007451 -0.018465  0.020042 -0.131481   
4     0.045241  0.059306      -0.010557 -0.022524  0.025989 -0.184079   
...        ...       ...            ...       ...       ...       ...   
5620 -0.818537 -0.779765       0.117596 -0.038906  0.055378 -0.744715   
5621 -0.818533 -0.779784       0.117650 -0.038906  0.055398 -0.744683   
5622 -0.818530 -0.779801       0.117697 -0.038906  0.055417 -0.744653   
5623 -0.818526 -0.779817       0.117739 -0.038906  0.055434 -0.744623   
5624 -0.818522 -0.779831       0.117775 -0.038905  0.055450 -0.744595   

         G_Lon  
0     0.004835  
1     0.017813  
2     0.038941  
3     0.067125  
4     0.100591  
...        ...  
5620